In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *

In [2]:
global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 128
model_name = "VGG16_quant4bit"
model = VGG16_quant()

conv_layer_index = 27

model.features[24] = QuantConv2d(256, 8, kernel_size=3, stride=1, padding=1, bias=False)
model.features[25] = nn.BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
model.features[conv_layer_index] = QuantConv2d(
    in_channels=8,
    out_channels=8,
    kernel_size=3,
    stride=1,
    padding=1,
    bias=False
)
model.features[28] = nn.Sequential()
model.features[30] = QuantConv2d(8, 512, kernel_size=3, stride=1, padding=1, bias=False)

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
VGG_quant(
  (features): Sequential(
    (0): QuantConv2d(
      3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): QuantConv2d(
      64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): QuantConv2d(
      64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (weight_quant): weight_quantize_fn()
    )
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): QuantConv2d(
      128, 128, kernel_size=(3, 3), stride

In [2]:
# This cell won't be given, but students will complete the training

lr = 3.95e-2
weight_decay = 1e-4
epochs = 200
best_prec = 0

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.92, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 3.483 (3.483)	Data 0.968 (0.968)	Loss 2.4381 (2.4381)	Prec 7.812% (7.812%)
Epoch: [0][100/391]	Time 0.043 (0.079)	Data 0.002 (0.013)	Loss 2.3082 (3.5804)	Prec 9.375% (11.409%)
Epoch: [0][200/391]	Time 0.049 (0.062)	Data 0.003 (0.008)	Loss 2.3215 (2.9492)	Prec 14.062% (11.785%)
Epoch: [0][300/391]	Time 0.042 (0.056)	Data 0.003 (0.006)	Loss 2.2700 (2.7269)	Prec 11.719% (12.059%)
Validation starts
Test: [0/79]	Time 0.291 (0.291)	Loss 2.2449 (2.2449)	Prec 14.062% (14.062%)
 * Prec 15.660% 
best acc: 15.660000
Epoch: [1][0/391]	Time 1.131 (1.131)	Data 1.074 (1.074)	Loss 2.3365 (2.3365)	Prec 13.281% (13.281%)
Epoch: [1][100/391]	Time 0.041 (0.052)	Data 0.002 (0.013)	Loss 2.2836 (2.2461)	Prec 18.750% (14.124%)
Epoch: [1][200/391]	Time 0.041 (0.047)	Data 0.002 (0.008)	Loss 2.2234 (2.2306)	Prec 17.188% (15.295%)
Epoch: [1][300/391]	Time 0.044 (0.046)	Data 0.003 (0.006)	Loss 2.1250 (2.2139)	Prec 21.094% (15.900%)
Validation starts
Test: [0/79]	Time 0.309 (0.309)	Loss 1.956

In [3]:
PATH = "result/VGG16_quant4bit/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9181/10000 (92%)



In [4]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []  
        
######### Save inputs from selected layer ##########
save_output = SaveOutput()
device = torch.device("cuda" if use_gpu else "cpu") 
counter =0
for layer in model.modules():
    if isinstance(layer, torch.nn.Conv2d):
        print("prehooked")
        counter += 1
        print(layer, counter)
        layer.register_forward_pre_hook(save_output)       ## Input for the module will be grapped       
####################################################

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.to(device)
out = model(images)


prehooked
QuantConv2d(
  3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 1
prehooked
QuantConv2d(
  64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 2
prehooked
QuantConv2d(
  64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 3
prehooked
QuantConv2d(
  128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 4
prehooked
QuantConv2d(
  128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 5
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 6
prehooked
QuantConv2d(
  256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
) 7
prehooked
QuantConv2d(
  2

In [5]:
weight = model.features[0].weight.abs().sum()
print(weight)

tensor(289.2113, device='cuda:0', grad_fn=<SumBackward0>)


In [6]:
w_bit = 4
weight_q = model.features[27].weight_q # quantized value is stored during the training
w_alpha = model.features[27].weight_quant.wgt_alpha
w_delta = w_alpha/(2**(w_bit-1)-1)
weight_int = weight_q/w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-7.0000, -2.0000, -1.0000],
          [-3.0000,  7.0000,  1.0000],
          [-6.0000,  7.0000,  3.0000]],

         [[ 3.0000,  3.0000,  2.0000],
          [ 2.0000,  3.0000,  2.0000],
          [ 1.0000,  1.0000,  1.0000]],

         [[ 7.0000,  1.0000,  1.0000],
          [ 2.0000, -1.0000, -5.0000],
          [ 2.0000, -4.0000, -4.0000]],

         [[-2.0000,  7.0000,  5.0000],
          [-0.0000, -2.0000,  4.0000],
          [ 1.0000,  3.0000, -1.0000]],

         [[-3.0000, -7.0000,  4.0000],
          [ 7.0000, -5.0000, -1.0000],
          [-4.0000, -0.0000,  3.0000]],

         [[-3.0000, -7.0000,  1.0000],
          [-1.0000, -3.0000, -1.0000],
          [-1.0000, -4.0000, -3.0000]],

         [[ 7.0000,  7.0000, -7.0000],
          [ 7.0000,  7.0000,  7.0000],
          [ 7.0000,  1.0000, -7.0000]],

         [[ 2.0000,  2.0000,  2.0000],
          [ 1.0000,  2.0000,  2.0000],
          [ 1.0000,  1.0000,  1.0000]]],


        [[[ 2.0000,  3.0000,  3.0000],
       

In [7]:
x_bit = 4
x = save_output.outputs[8][0]  # input of the 8th conv layer
x_alpha  = model.features[27].act_alpha
x_delta = x_alpha/(2**x_bit-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta
print(x_int) # you should see clean integer numbers 

tensor([[[[ 2.0000,  0.0000,  3.0000,  5.0000],
          [ 1.0000,  0.0000,  1.0000,  3.0000],
          [ 2.0000,  0.0000,  2.0000,  3.0000],
          [ 5.0000,  3.0000,  3.0000,  5.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 3.0000,  2.0000,  3.0000,  1.0000],
          [ 0.0000,  0.0000,  1.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         ...,

         [[ 0.0000,  1.0000,  1.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 1.0000,  1.0000,  0.0000,  0.0000]],

         [[ 4.0000,  4.0000,  8.0000,  5.0000],
          [ 5.0000,  1.0000,  4.0000,  1.0000],
          [ 7.0000,  3.0000,  6.0000,  4.0000],
          [ 9.0000,  6.0000,  8.0000,  6.0000]],

         [[ 0.0

In [8]:
conv_int = torch.nn.Conv2d(in_channels = 8, out_channels=8, kernel_size = 3, padding=1, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)
relu = nn.ReLU()
output_int = conv_int(x_int)
output_recovered = output_int*w_delta*x_delta
output_recovered = relu(output_recovered)

In [9]:
difference = abs(save_output.outputs[9][0] - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(2.2577e-07, device='cuda:0', grad_fn=<MeanBackward0>)


In [10]:
x_pad = torch.zeros(128, 8, 6, 6).cuda()

In [11]:
x_pad[ : ,  :, 1:5, 1:5] = x_int.cuda()


In [12]:
X = x_pad[0]
X = torch.reshape(X, (X.size(0), -1))

In [13]:
X.size()

torch.Size([8, 36])

In [14]:
tile_id = 0 
nij = 200 # just a random number
#X = a_tile[tile_id,:,nij:nij+64]  # [tile_num, array row num, time_steps]

bit_precision = 4
file = open('activation.txt', 'w') #write to file
file.write('#time0row7[msb-lsb],time0row6[msb-lst],....,time0row0[msb-lst]#\n')
file.write('#time1row7[msb-lsb],time1row6[msb-lst],....,time1row0[msb-lst]#\n')
file.write('#................#\n')

for i in range(X.size(1)):  # time step
    for j in range(X.size(0)): # row #
        X_bin = '{0:04b}'.format(int(X[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(X_bin[k])        
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close() #close file 

In [15]:
X[:,7]

tensor([2.0000, 0.0000, 3.0000, 6.0000, 2.0000, 0.0000, 4.0000, 0.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [16]:
weight_int.size()

torch.Size([8, 8, 3, 3])

In [17]:
weight_int.size() # 8, 8 , 3, 3
W = torch.reshape(weight_int, (weight_int.size(0), weight_int.size(1), -1))
W.size() # 8, 8, 9

torch.Size([8, 8, 9])

In [18]:
bit_precision = 4
file = open('weight.txt', 'w') #write to file
file.write('#col0row7[msb-lsb],col0row6[msb-lst],....,col0row0[msb-lst]#\n')
file.write('#col1row7[msb-lsb],col1row6[msb-lst],....,col1row0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(W.size(0)):  
        for j in range(W.size(1)):
            if (W[i, 7-j, kij].item()<0):
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+2**bit_precision+0.001))
            else:
                W_bin = '{0:04b}'.format(int(W[i,7-j,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(W_bin[k])        
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close() #close file  


In [19]:
W[0,:,0]

tensor([-7.0000,  3.0000,  7.0000, -2.0000, -3.0000, -3.0000,  7.0000,  2.0000],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [20]:
p_nijg = range(X.size(1)) ## psum nij group

In [21]:
psum = torch.zeros(8, len(p_nijg), 9).cuda() 

In [22]:
for kij in range(9):  
    for nij in p_nijg:       # time domain, sequentially given input
        m = nn.Linear(8, 8, bias=False)
        m.weight = torch.nn.Parameter(W[:,:,kij])
        psum[:, nij, kij] = m(X[:,nij]).cuda()

In [23]:
bit_precision = 16
file = open('psum.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')
for kij in range(9):
    for i in range(psum.size(1)):
        for j in range(psum.size(0)):
            if (psum[7-j,i,kij].item()<0):
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+2**bit_precision+0.001))
            else:
                P_bin = '{0:016b}'.format(int(psum[7-j,i,kij].item()+0.001))
            for k in range(bit_precision):
                file.write(P_bin[k])
            #file.write(' ')  # for visibility with blank between words, you can use
        file.write('\n')
file.close()

In [24]:
psum[:,8,0]

tensor([ 25.,   7.,  13., -21., -78., -34., -11.,  53.], device='cuda:0',
       grad_fn=<SelectBackward0>)

In [25]:
output_int.size()

torch.Size([128, 8, 4, 4])

In [26]:
out = output_int[0]

In [27]:
out = torch.reshape(out, (out.size(0), -1))

In [28]:
out.size()

torch.Size([8, 16])

In [29]:
bit_precision = 16
file = open('output.txt', 'w') #write to file
file.write('#time0col7[msb-lsb],time0col6[msb-lst],....,time0col0[msb-lst]#\n')
file.write('#time1col7[msb-lsb],time1col6[msb-lst],....,time1col0[msb-lst]#\n')
file.write('#................#\n')

for i in range(out.size(1)):
    for j in range(out.size(0)):
        if (out[7-j,i].item()<0):
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+2**bit_precision+0.001))
        else:
            O_bin = '{0:016b}'.format(int(out[7-j,i].item()+0.001))
        for k in range(bit_precision):
            file.write(O_bin[k])
        #file.write(' ')  # for visibility with blank between words, you can use
    file.write('\n')
file.close()

In [30]:
out[:,4]

tensor([ 128.0000,  -92.0000,   39.0000,  -16.0000, -142.0000,  -17.0000,
         237.0000,   65.0000], device='cuda:0', grad_fn=<SelectBackward0>)

In [5]:
 import torch.nn.utils.prune as prune

In [6]:
# Structure Pruning
for layer in model.modules():
    if isinstance(layer, QuantConv2d):
        prune.ln_structured(layer, name='weight', amount=0.8, n=1, dim=0)

In [7]:
for layer in model.modules():
    if isinstance(layer, QuantConv2d):
        mask1 = layer.weight_mask
        sparsity_mask1 = (mask1 == 0).sum() / mask1.nelement()
        print("Sparsity level: ", sparsity_mask1)

Sparsity level:  tensor(0.7969, device='cuda:0')
Sparsity level:  tensor(0.7969, device='cuda:0')
Sparsity level:  tensor(0.7969, device='cuda:0')
Sparsity level:  tensor(0.7969, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')
Sparsity level:  tensor(0.7500, device='cuda:0')
Sparsity level:  tensor(0.7500, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')
Sparsity level:  tensor(0.8008, device='cuda:0')


In [8]:
model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 1000/10000 (10%)



In [10]:
# Training after pruning
lr = 3.95e-2
weight_decay = 1e-4
epochs = 200
best_prec = 0

model_name = 'pruning_VGG16'

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.92, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

Epoch: [0][0/391]	Time 0.192 (0.192)	Data 0.154 (0.154)	Loss 1.1248 (1.1248)	Prec 55.469% (55.469%)
Epoch: [0][100/391]	Time 0.043 (0.046)	Data 0.002 (0.003)	Loss 1.0460 (0.9851)	Prec 59.375% (64.898%)
Epoch: [0][200/391]	Time 0.042 (0.045)	Data 0.002 (0.003)	Loss 0.9722 (0.9709)	Prec 64.844% (65.485%)
Epoch: [0][300/391]	Time 0.047 (0.045)	Data 0.002 (0.002)	Loss 0.9002 (0.9729)	Prec 67.188% (65.425%)
Validation starts
Test: [0/79]	Time 0.213 (0.213)	Loss 0.9991 (0.9991)	Prec 62.500% (62.500%)
 * Prec 64.780% 
best acc: 64.780000
Epoch: [1][0/391]	Time 0.234 (0.234)	Data 0.189 (0.189)	Loss 1.0557 (1.0557)	Prec 66.406% (66.406%)
Epoch: [1][100/391]	Time 0.045 (0.046)	Data 0.002 (0.004)	Loss 0.9652 (0.9649)	Prec 67.969% (65.524%)
Epoch: [1][200/391]	Time 0.043 (0.045)	Data 0.002 (0.003)	Loss 0.8316 (0.9506)	Prec 69.531% (66.084%)
Epoch: [1][300/391]	Time 0.043 (0.045)	Data 0.002 (0.002)	Loss 1.1129 (0.9480)	Prec 60.156% (66.157%)
Validation starts
Test: [0/79]	Time 0.174 (0.174)	Loss 0.

In [11]:
model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 7892/10000 (79%)

